In [1]:
import json
import numpy as np
import heapq
import math

In [2]:
# Downloading JSON files
with open('Coord.json') as json_file:
    coord = json.load(json_file)
with open('Cost.json') as json_file:
    cost = json.load(json_file)
with open('Dist.json') as json_file:
    dist = json.load(json_file)
with open('G.json') as json_file:
    g = json.load(json_file)

In [3]:
len(g)

264346

In [4]:
#Initializing start and end
START="1"
END="50"

# Task 1: Use any algorithmn without the energy constraint

In [5]:
#Can ignore Cost and just focus on Dist
#Since solution quality matters, probably should use an informed search algorithm

#A simple heuristic function will be to find the distance of coord to end goal using pythagores

#Another way to make it more complex is to multiple that distance with the estimated cost of moving
#that distance using its neighbours as reference

#https://stackoverflow.com/questions/22898122/heuristic-function-for-shortest-path
#https://python.plainenglish.io/uniform-cost-search-ucs-algorithm-in-python-ec3ee03fca9f

In [6]:
class Node:
    def __init__(self,index,parent):
        self.index= index
        self.coords=coord[index]
        self.neighbours=g[index]
        self.heuristic=np.inf
        self.parent = parent
        if parent != None:
            self.calcHeuristic(self.parent)
        else:
            self.distance=0
        self.explored=False
    
    def __gt__(self,other):
        if isinstance(other,Node):
            if self.heuristic>other.heuristic:
                return True
            if self.heuristic<other.heuristic:
                return False
            return self.index>other.index #Rank by index if same heuristic
        
    def explore(self):
        self.explored=True
        if self.parent==None:
            self.path=self.index
            self.pathLength=1
        else:
            self.path=self.parent.getPath() + "," + self.index
            self.pathLength=self.parent.getPathLength()+1
    
    def getCoords(self):
        return coord[index]

    def getNeighbours(self):
        return self.neighbours
    
    def calcHeuristic(self,parent):
        parentIndex=parent.getIndex()
        distanceFromStart=parent.getDistance()+dist[self.index+","+parentIndex]
        if(len(self.neighbours)==1):
            return False
        x2=self.coords[0]-coord[END][0]
        y2=self.coords[1]-coord[END][1]
        distanceFromEnd=math.sqrt(x2**2+y2**2)
        heuristic=distanceFromStart+distanceFromEnd
        if(heuristic>self.heuristic):
            return False
        else:
            self.heuristic=heuristic
            self.setParent(parent)
            self.distance=distanceFromStart
            return True

    def getPath(self):
        return self.path
    
    def getPathLength(self):
        return self.pathLength
    
    def setParent(self,parent):
        self.parent=parent
    
    def getDistance(self):
        return self.distance
    
    def getIndex(self):
        return self.index

In [7]:
def search():
    start= Node(START,None)
    pq=[start]
    dictOfNodes=dict()
    dictOfNodes[start.index]=start
    nodesExplored=0
    while len(pq) > 0:
        nextExplore=heapq.heappop(pq)
        nextExplore.explore()
        nodesExplored+=1
        if nextExplore.index==END:
            print("Nodes explored: ",nodesExplored)
            return nextExplore.getPath(),nextExplore.getPathLength(),nextExplore.getDistance()
        new_nodes=nextExplore.getNeighbours()
        if len(new_nodes)>1:
            for new_node in new_nodes:
                if new_node in dictOfNodes:
                    tempNode=dictOfNodes.get(new_node)
                    if(tempNode.explored==False):
                        tempNode.calcHeuristic(nextExplore)
                else:
                    temp= Node(new_node,nextExplore)
                    dictOfNodes[temp.index]=temp
                    heapq.heappush(pq, temp)
    return "No path found"

In [8]:
path,length,distance=search()

Nodes explored:  968


In [9]:
print(path)

1,1363,1358,1357,1356,1276,1273,1277,1269,1267,1268,1284,1283,1282,1255,1253,1260,1259,1249,1246,963,964,962,1002,952,1000,998,994,995,996,987,986,979,980,969,977,989,990,991,2369,2366,2340,2338,2339,2333,2334,2329,2029,2027,2019,2022,2000,1996,1997,1993,1992,1989,1984,2001,1900,1875,1874,1965,1963,1964,1923,1944,1945,1938,1937,1939,1935,1931,1934,1673,1675,1674,1837,1671,1828,1825,1817,1815,1634,1814,1813,1632,1631,1742,1741,1740,1739,1591,1689,1585,1584,1688,1579,1679,1677,104,5680,5418,5431,5425,5424,5422,5413,5412,5411,66,5392,5391,5388,5291,5278,5289,5290,5283,5284,5280,50


In [10]:
temp=path.split(",")
len(temp)

122

In [11]:
print(length)

122


In [12]:
print(distance)

148648.63722140007


# Task 2: Uninformed search algorithm taking into account the energy budget

In [13]:
ENERGY_BUDGET=287932

In [14]:
class Node2:
    def __init__(self,index,parent):
        self.index= index
        self.coords=coord[index]
        self.neighbours=g[index]
        self.distance=np.inf
        self.heuristic=np.inf
        self.cost=np.inf
        self.parent = parent
        if parent != None:
            self.calcHeuristic(self.parent)
        else:
            self.distance=0
            self.cost=0
        self.explored=False
    
    def __gt__(self,other):
        if isinstance(other,Node2):
            if self.heuristic>other.heuristic:
                return True
            if self.heuristic<other.heuristic:
                return False
            return self.cost>other.cost #Rank by index if same heuristic
        
    def explore(self):
        self.explored=True
        if self.parent==None:
            self.path=self.index
            self.pathLength=1
        else:
            self.path=self.parent.getPath() + "," + self.index
            self.pathLength=self.parent.getPathLength()+1
    
    def getCoords(self):
        return coord[index]

    def getNeighbours(self):
        return self.neighbours
    
    def calcHeuristic(self,parent):
        parentIndex=parent.getIndex()
        distanceFromStart=parent.getDistance()+dist[self.index+","+parentIndex]
        if(len(self.neighbours)==1):
            return False
        total_cost=parent.getCost()+ cost[self.index+","+parent.getIndex()]
        heuristic=distanceFromStart+total_cost*0.5
        if(distanceFromStart<self.distance):
            self.heuristic=heuristic
            self.setParent(parent)
            self.distance=distanceFromStart
            self.cost= total_cost
            return True
        else:
            return False
        
    def checkCost(self,parent):
        return (parent.getCost()+ cost[self.index+","+parent.getIndex()]) <= ENERGY_BUDGET

    def getPath(self):
        return self.path
    
    def getCost(self):
        return self.cost
    
    def getPathLength(self):
        return self.pathLength
    
    def setParent(self,parent):
        self.parent=parent
    
    def getDistance(self):
        return self.distance
    
    def getIndex(self):
        return self.index

In [15]:
def search2():
    start= Node2(START,None)
    pq=[start]
    dictOfNodes=dict()
    dictOfNodes[start.index]=start
    nodesExplored=0
    while len(pq) > 0:
        nextExplore=heapq.heappop(pq)
        nextExplore.explore()
        nodesExplored+=1
        if nextExplore.index==END:
            print("Nodes explored: ",nodesExplored)
            return nextExplore.getPath(),nextExplore.getPathLength(),nextExplore.getDistance(),nextExplore.getCost()
        new_nodes=nextExplore.getNeighbours()
        if len(new_nodes)>1:
            for new_node in new_nodes:
                if new_node in dictOfNodes:
                    tempNode=dictOfNodes.get(new_node)
                    if(tempNode.checkCost(nextExplore)):
                        tempNode.calcHeuristic(nextExplore)
                else:
                    temp= Node2(new_node,nextExplore)
                    if(temp.checkCost(nextExplore)):
                        dictOfNodes[temp.index]=temp
                        heapq.heappush(pq, temp)
    return "No path found"

In [16]:
path,length,distance,totalcost=search2()

Nodes explored:  4051


In [17]:
path

'1,1363,1358,1357,1356,1276,1273,1277,1269,1267,1268,1284,1283,1282,1255,1253,1260,1259,1249,1246,963,964,962,1002,952,1000,998,994,995,996,987,988,979,980,969,977,989,990,991,2465,2466,2384,2382,2385,2379,2380,2445,2444,2405,2406,2398,2395,2397,2142,2141,2125,2126,2082,2080,2071,1979,1975,1967,1966,1974,1973,1971,1970,1948,1937,1939,1935,1931,1934,1673,1675,1674,1837,1671,1828,1825,1817,1815,1634,1814,1813,1632,1631,1742,1741,1740,1739,1591,1689,1585,1584,1688,1579,1679,1677,104,5680,5418,5431,5425,5424,5422,5413,5412,5411,66,5392,5391,5388,5291,5278,5289,5290,5283,5284,5280,50'

In [18]:
length

122

In [19]:
distance

150335.55441905273

In [20]:
totalcost

259087

# Task 3: Informed search algorithm taking into account the energy budget

In [21]:
lowestCostPerDistance=np.inf
for i in cost:
    tempCost=cost[i]/dist[i]
    if tempCost<lowestCostPerDistance:
        lowestCostPerDistance=tempCost
print(lowestCostPerDistance)

0.8235294117647058


In [35]:
class Node3:
    def __init__(self,index,parent):
        self.index= index
        self.coords=coord[index]
        self.neighbours=g[index]
        self.heuristic=np.inf
        self.cost=np.inf
        self.parent = parent
        if parent != None:
            self.calcHeuristic(self.parent)
        else:
            self.distance=0
            self.cost=0
        self.explored=False
    
    def __gt__(self,other):
        if isinstance(other,Node3):
            if self.heuristic>other.heuristic:
                return True
            if self.heuristic<other.heuristic:
                return False
            return self.cost>other.cost #Rank by index if same heuristic
        
    def explore(self):
        self.explored=True
        if self.parent==None:
            self.path=self.index
            self.pathLength=1
        else:
            self.path=self.parent.getPath() + "," + self.index
            self.pathLength=self.parent.getPathLength()+1
    
    def getCoords(self):
        return coord[index]

    def getNeighbours(self):
        return self.neighbours
    
    def calcHeuristic(self,parent):
        parentIndex=parent.getIndex()
        distanceFromStart=parent.getDistance()+dist[self.index+","+parentIndex]
        if(len(self.neighbours)==1):
            return False
        heuristic=distanceFromStart
        x2=self.coords[0]-coord[END][0]
        y2=self.coords[1]-coord[END][1]
        distanceFromEnd=math.sqrt(x2**2+y2**2)
        total_cost=parent.getCost()+ cost[self.index+","+parent.getIndex()]
        heuristic=distanceFromStart+distanceFromEnd+total_cost*0.05
        if(heuristic>self.heuristic):
            return False
        else:
            self.heuristic=heuristic
            self.setParent(parent)
            self.distance=distanceFromStart
            self.cost= total_cost
            return True
        
    def checkCost(self,parent):
        x2=self.coords[0]-coord[END][0]
        y2=self.coords[1]-coord[END][1]
        distanceFromEnd=math.sqrt(x2**2+y2**2)
        return (parent.getCost()+ cost[self.index+","+parent.getIndex()] + distanceFromEnd*lowestCostPerDistance) <= ENERGY_BUDGET

    def getPath(self):
        return self.path
    
    def getCost(self):
        return self.cost
    
    def getPathLength(self):
        return self.pathLength
    
    def setParent(self,parent):
        self.parent=parent
    
    def getDistance(self):
        return self.distance
    
    def getIndex(self):
        return self.index

In [51]:
def search3():
    start= Node3(START,None)
    pq=[start]
    dictOfNodes=dict()
    dictOfNodes[start.index]=start
    nodesExplored=0
    while len(pq) > 0:
        nextExplore=heapq.heappop(pq)
        nextExplore.explore()
        nodesExplored+=1
        if nextExplore.index==END:
            print("Nodes explored: ",nodesExplored)
            return nextExplore.getPath(),nextExplore.getPathLength(),nextExplore.getDistance(),nextExplore.getCost()
        new_nodes=nextExplore.getNeighbours()
        if len(new_nodes)>1:
            for new_node in new_nodes:
                if new_node in dictOfNodes:
                    tempNode=dictOfNodes.get(new_node)
                    if(tempNode.explored==False):
                        if(tempNode.checkCost(nextExplore)):
                            tempNode.calcHeuristic(nextExplore)
                            heapq.heapify(pq)
                else:
                    temp= Node3(new_node,nextExplore)
                    if(temp.checkCost(nextExplore)):
                        dictOfNodes[temp.index]=temp
                        heapq.heappush(pq, temp)
    return "No path found"

In [52]:
path,length,distance,totalcost=search3()

Nodes explored:  1647


In [53]:
path

'1,1363,1358,1357,1356,1276,1273,1277,1269,1267,1268,1284,1283,1282,1255,1253,1260,1259,1249,1246,963,964,962,1002,952,1000,998,994,995,996,987,986,979,980,969,977,989,990,991,2465,2466,2384,2382,2385,2379,2380,2445,2444,2405,2406,2398,2395,2397,2142,2141,2125,2126,2082,2080,2071,1979,1975,1967,1966,1974,1973,1971,1970,1948,1937,1939,1935,1931,1934,1673,1675,1674,1837,1671,1828,1825,1817,1815,1634,1814,1813,1632,1631,1742,1741,1740,1739,1591,1689,1585,1584,1688,1579,1679,1677,104,5680,5418,5431,5425,5424,5422,5413,5412,5411,66,5392,5391,5388,5291,5278,5289,5290,5283,5284,5280,50'

In [54]:
length

122

In [55]:
distance

150335.55441905273

In [56]:
totalcost

259087